In [1]:
import os, shutil, sys, warnings, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from zipfile import ZipFile
from tqdm import trange
from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score
import tensorflow
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
# Ignore all warnings
warnings.simplefilter("ignore")

In [23]:
CONFIG = {
    "TEST_FOLDER" : "kaggle_level_2",
    "TRAIN_FOLDER" : "level_2/",
    "IMAGE_SIZE": (16, 16),
    "THRESHOLD": 0.5
}

In [24]:
def read_all(folder_path, key_prefix=""):
    print("Reading:")
    images={}
    files = os.listdir(folder_path)
    for i, filename in tqdm(enumerate(files), total=len(files)):
        file_path = os.path.join(folder_path,filename)
        image_index = key_prefix + filename[:-4]
        image = Image.open(file_path)
        image = image.convert("L")
        images[image_index] = np.array(image.copy()).flatten()
        image.close()
    return images

In [25]:
languages = ['ta', 'hi', 'en'] # tamil, hindi, english
# read in backgrounds with extra 'bgr_' for identification
images_train = read_all(f'{CONFIG["TRAIN_FOLDER"]}/background/',key_prefix="bgr_") # change the path
# read in the images from all three languages with extra prefix for identification
for language in languages:
  images_train.update(read_all(CONFIG["TRAIN_FOLDER"] + '/' + language, key_prefix=language+"_" ))
print(len(images_train))

# read in test images
images_test = read_all(CONFIG["TEST_FOLDER"], key_prefix='') # change the path
print(len(images_test))

list(images_test.keys())[:5]

X = []
Y = []

Reading:


  0%|          | 0/407 [00:00<?, ?it/s]

Reading:


  0%|          | 0/114 [00:00<?, ?it/s]

Reading:


...

Reading:


  0%|          | 0/146 [00:00<?, ?it/s]

803
Reading:


  0%|          | 0/299 [00:00<?, ?it/s]

299


In [26]:
# add the image to columns then check if it is background image or not, depending on that put 0 or 1 in target
for key, value in images_train.items():
    X.append(value)
    if key[:4] == "bgr_":
        Y.append(0)
    else:
        Y.append(1)

ID_test = []
X_test = []
for key, value in images_test.items():
  ID_test.append(int(key))
  X_test.append(value)
  
        
X = np.array(X)
Y = np.array(Y)
X_test = np.array(X_test)

print(X.shape, Y.shape)
print(X_test.shape)

(803, 256) (803,)
(299, 256)


In [27]:
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [28]:
# splitting for train and validation sets to check bias chances on test data. we already have the test data
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.15, random_state=10, stratify=Y, shuffle=True)

In [29]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [30]:
# Creating a simple neural network model
model = Sequential()
model.add(Dense(10, input_shape=(256,), activation='sigmoid'))  # Input layer with 10 neurons
model.add(Dense(4,activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))  # Output layer with 1 neuron for binary classification

# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [31]:
# Training the model
model.fit(X_train,Y_train, epochs=25, batch_size=5,validation_split=0.2)

Epoch 1/25
109/109 [==============================] - 2s 8ms/step - loss: 0.6380 - accuracy: 0.5064 - val_loss: 0.5485 - val_accuracy: 0.5109
Epoch 2/25
109/109 [==============================] - 1s 5ms/step - loss: 0.4962 - accuracy: 0.5064 - val_loss: 0.4772 - val_accuracy: 0.5109
Epoch 3/25
109/109 [==============================] - 1s 5ms/step - loss: 0.4346 - accuracy: 0.6752 - val_loss: 0.4271 - val_accuracy: 0.9051
Epoch 4/25
109/109 [==============================] - 1s 5ms/step - loss: 0.3883 - accuracy: 0.9468 - val_loss: 0.3884 - val_accuracy: 0.9343
Epoch 5/25
109/109 [==============================] - 1s 5ms/step - loss: 0.3507 - accuracy: 0.9743 - val_loss: 0.3573 - val_accuracy: 0.9416
Epoch 6/25
109/109 [==============================] - 1s 6ms/step - loss: 0.3193 - accuracy: 0.9872 - val_loss: 0.3311 - val_accuracy: 0.9562
Epoch 7/25
109/109 [==============================] - 1s 6ms/step - loss: 0.2930 - accuracy: 0.9908 - val_loss: 0.3073 - val_accuracy: 0.9562
Epoch 

In [32]:
# Assuming you have X_test and y_test as your test dataset

# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(X_val, Y_val)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

4/4 [==============================] - 0s 6ms/step - loss: 0.1003 - accuracy: 0.9917
Test Loss: 0.1003
Test Accuracy: 0.9917
